In [11]:
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
import os
import pandas as pd

# Path ke folder yang berisi file-file DataFrame
folder_path = '../comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'

# Dictionary untuk menyimpan semua DataFrame dengan nama file sebagai key
df_dict = {}

# Loop melalui semua file dalam folder
for file_name in os.listdir(folder_path):
    # Pastikan file yang dibaca adalah file yang diinginkan (misalnya, CSV)
    if file_name.endswith('.csv'):
        # Buat path lengkap ke file
        file_path = os.path.join(folder_path, file_name)
        
        # Baca file dan simpan ke dictionary dengan nama file sebagai key
        df = pd.read_csv(file_path)
        df_dict[file_name] = df  # Gunakan nama file sebagai key


In [13]:
df_bawang_merah = df_dict['Bawang Merah.csv']
df_bawang_putih = df_dict['Bawang Putih Bonggol.csv']
df_beras_medium = df_dict['Beras Medium.csv']
df_beras_premium = df_dict['Beras Premium.csv']
df_cabai_merah_keriting = df_dict['Cabai Merah Keriting.csv']
df_cabai_rawit_merah = df_dict['Cabai Rawit Merah.csv']
df_daging_ayam = df_dict['Daging Ayam Ras.csv']
df_daging_sapi = df_dict['Daging Sapi Murni.csv']
df_gula = df_dict['Gula Konsumsi.csv']
df_minyak_goreng_curah = df_dict['Minyak Goreng Curah.csv']
df_minyak_goreng_kemasan = df_dict['Minyak Goreng Kemasan Sederhana.csv']
df_telur_ayam = df_dict['Telur Ayam Ras.csv']
df_tepung_terigu = df_dict['Tepung Terigu (Curah).csv']

In [14]:
import pandas as pd

# List semua dataframe
df_list = [
    df_bawang_merah, df_bawang_putih, df_beras_medium, df_beras_premium, df_cabai_merah_keriting,
    df_cabai_rawit_merah, df_daging_ayam, df_daging_sapi, df_gula, df_minyak_goreng_curah,
    df_minyak_goreng_kemasan, df_telur_ayam, df_tepung_terigu
]

# Loop setiap dataframe dan lakukan interpolasi
for i in range(len(df_list)):
    df = df_list[i]  # Ambil dataframe

    # Pisahkan kolom tanggal (asumsi ada kolom 'date' atau 'tanggal')
    date_cols = df.select_dtypes(include=['datetime64', 'object'])  # Deteksi kolom tanggal
    num_cols = df.select_dtypes(include=['number'])  # Ambil hanya kolom numerik

    # Interpolasi hanya untuk kolom numerik
    df_num_imputed = num_cols.interpolate(method='linear', limit_direction='both')

    # Gabungkan kembali kolom tanggal dengan hasil interpolasi
    df_list[i] = pd.concat([date_cols.reset_index(drop=True), df_num_imputed.reset_index(drop=True)], axis=1)


In [15]:
df_dict = {
    'Bawang Merah': df_list[0],
    'Bawang Putih Bonggol': df_list[1],
    'Beras Medium': df_list[2],
    'Beras Premium': df_list[3],
    'Cabai Merah Keriting': df_list[4],
    'Cabai Rawit Merah': df_list[5],
    'Daging Ayam Ras': df_list[6],
    'Daging Sapi Murni': df_list[7],
    'Gula Konsumsi': df_list[8],
    'Minyak Goreng Curah': df_list[9],
    'Minyak Goreng Kemasan Sederhana': df_list[10],
    'Telur Ayam Ras': df_list[11],
    'Tepung Terigu (Curah)': df_list[12]
}


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Parameter
seq_length = 30  # Panjang sequence
learning_rate = 0.001
epochs = 100
# Rentang waktu prediksi
predict_start = '2024-10-01'
predict_end = '2024-12-31'
predict_periods = (pd.to_datetime(predict_end) - pd.to_datetime(predict_start)).days + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [17]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size * 2)  # BiLSTM output
        self.v = nn.Parameter(torch.rand(hidden_size * 2))  # Attention weights

    def forward(self, lstm_output):
        # Compute attention scores
        attn_weights = torch.tanh(self.attn(lstm_output))  # (batch, seq_len, hidden_size * 2)
        attn_weights = torch.matmul(attn_weights, self.v)  # (batch, seq_len)
        attn_weights = torch.softmax(attn_weights, dim=1)  # Normalize

        # Apply attention weights
        context = torch.sum(lstm_output * attn_weights.unsqueeze(-1), dim=1)  # (batch, hidden_size * 2)
        return context

In [18]:
class BiLSTMPredictorWithAttention(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3):
        super(BiLSTMPredictorWithAttention, self).__init__()

        # BiLSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=True)

        # Attention Layer
        self.attention = Attention(hidden_size)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden_size * 2)
        attn_out = self.attention(lstm_out)  # (batch, hidden_size * 2)
        out = self.fc(attn_out)  # (batch, output_size)
        return out.unsqueeze(-1)  # Ensure output shape (batch_size, 1, 1)

In [19]:
# Direktori penyimpanan model terbaik
if not os.path.exists("best_models_bilstm_attention"): os.makedirs("best_models_bilstm_attention")

# Menyimpan hasil prediksi dan MAPE terbaik
all_predictions = []
mape_results = {}

In [20]:
for komoditas, df_komoditas in df_dict.items():
    print(f"\n=== Memproses {komoditas} ===")

    df_komoditas['Date'] = pd.to_datetime(df_komoditas['Date'])
    
    for provinsi in df_komoditas.columns[1:]:
        print(f"  → Prediksi untuk {provinsi}...")

        df_prov = df_komoditas[['Date', provinsi]].rename(columns={'Date': 'ds', provinsi: 'y'})
        
        # Normalisasi data
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df_prov[['y']])
        
        X, y = [], []
        for i in range(len(df_scaled) - seq_length):
            X.append(df_scaled[i:i+seq_length])  # Menggunakan hanya harga komoditas sebagai input
            y.append(df_scaled[i+seq_length])  # Target harga komoditas
        
        X, y = np.array(X), np.array(y)
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)
        
        # Pisahkan training & testing
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Bangun model LSTM
        model = BiLSTMPredictorWithAttention(input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3).to(device)
        criterion = nn.SmoothL1Loss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
        
        best_val_loss = float("inf")
        best_mape = float("inf")
        best_model_path = f"best_models/univariate_{komoditas}_{provinsi}.pt"

        # Train model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    test_output = model(X_test)
                    val_loss = criterion(test_output, y_test).item()
                    
                    test_output_np = test_output.cpu().numpy().reshape(-1, 1)
                    y_test_np = y_test.cpu().numpy().reshape(-1, 1)

                    test_output_np = scaler.inverse_transform(test_output_np)
                    y_test_np = scaler.inverse_transform(y_test_np)

                    mape = mean_absolute_percentage_error(y_test_np, test_output_np)


                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.6f}, Validation Loss: {val_loss:.6f}, MAPE: {mape:.6f}")
                
                scheduler.step(val_loss)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_mape = mape
                    torch.save(model.state_dict(), best_model_path)
        
        mape_results[f"{komoditas}_{provinsi}"] = best_mape

        # Load model terbaik untuk prediksi masa depan
        best_model = BiLSTMPredictorWithAttention(input_size=1).to(device)
        best_model.load_state_dict(torch.load(best_model_path))
        best_model.eval()
        
        last_sequence = X[-1].unsqueeze(0)
        future_preds = []

        for _ in range(predict_periods):
            with torch.no_grad():
                next_pred = best_model(last_sequence).item()
            future_preds.append(next_pred)
            
            next_input = torch.tensor([[next_pred]], dtype=torch.float32).unsqueeze(0).to(device)
            last_sequence = torch.cat([last_sequence[:, 1:, :], next_input], dim=1)

        future_preds = np.array(future_preds).reshape(-1, 1)
        future_preds = scaler.inverse_transform(future_preds)

        for i, pred in enumerate(future_preds):
            date = pd.to_datetime(predict_start) + pd.Timedelta(days=i)
            all_predictions.append({
                'id': f"{komoditas}/{provinsi}/{date.strftime('%Y-%m-%d')}",
                'price': pred[0]
            })

# Simpan hasil prediksi ke CSV
submission_df = pd.DataFrame(all_predictions)
submission_df.to_csv('submission_univariate_bilstm_mape.csv', index=False)


=== Memproses Bawang Merah ===
  → Prediksi untuk Aceh...


/home/sirius/Documents/DS/myenv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 10/150, Training Loss: 0.014774, Validation Loss: 0.068221, MAPE: 0.259769
Epoch 20/150, Training Loss: 0.013052, Validation Loss: 0.044645, MAPE: 0.252920
Epoch 30/150, Training Loss: 0.009789, Validation Loss: 0.037443, MAPE: 0.209728
Epoch 40/150, Training Loss: 0.004975, Validation Loss: 0.015524, MAPE: 0.132533
Epoch 50/150, Training Loss: 0.003371, Validation Loss: 0.010452, MAPE: 0.102317
Epoch 60/150, Training Loss: 0.001337, Validation Loss: 0.003619, MAPE: 0.050347
Epoch 70/150, Training Loss: 0.001436, Validation Loss: 0.003479, MAPE: 0.077664
Epoch 80/150, Training Loss: 0.001226, Validation Loss: 0.002629, MAPE: 0.056842
Epoch 90/150, Training Loss: 0.001003, Validation Loss: 0.002166, MAPE: 0.041661
Epoch 100/150, Training Loss: 0.000798, Validation Loss: 0.002331, MAPE: 0.051851
Epoch 110/150, Training Loss: 0.000739, Validation Loss: 0.002077, MAPE: 0.041299
Epoch 120/150, Training Loss: 0.000701, Validation Loss: 0.002119, MAPE: 0.043200
Epoch 130/150, Training L

/tmp/ipykernel_170358/2597736274.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/150, Training Loss: 0.012928, Validation Loss: 0.031247, MAPE: 0.200928
Epoch 20/150, Training Loss: 0.012251, Validation Loss: 0.021531, MAPE: 0.222327
Epoch 30/150, Training Loss: 0.011493, Validation Loss: 0.023941, MAPE: 0.189805
Epoch 40/150, Training Loss: 0.009024, Validation Loss: 0.014393, MAPE: 0.173716
Epoch 50/150, Training Loss: 0.004869, Validation Loss: 0.006402, MAPE: 0.102063
Epoch 60/150, Training Loss: 0.003339, Validation Loss: 0.004505, MAPE: 0.081910
Epoch 70/150, Training Loss: 0.001694, Validation Loss: 0.002156, MAPE: 0.052072
Epoch 80/150, Training Loss: 0.001178, Validation Loss: 0.001633, MAPE: 0.050939
Epoch 90/150, Training Loss: 0.001030, Validation Loss: 0.001355, MAPE: 0.045390
Epoch 100/150, Training Loss: 0.000896, Validation Loss: 0.001343, MAPE: 0.052379
Epoch 110/150, Training Loss: 0.003201, Validation Loss: 0.002268, MAPE: 0.084238
Epoch 120/150, Training Loss: 0.001150, Validation Loss: 0.001745, MAPE: 0.042863
Epoch 130/150, Training L

...

/tmp/ipykernel_170358/2597736274.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/150, Training Loss: 0.012860, Validation Loss: 0.013557, MAPE: 0.051719
Epoch 20/150, Training Loss: 0.012169, Validation Loss: 0.046193, MAPE: 0.101006
Epoch 30/150, Training Loss: 0.011829, Validation Loss: 0.014147, MAPE: 0.053872
Epoch 40/150, Training Loss: 0.007991, Validation Loss: 0.025558, MAPE: 0.075192
Epoch 50/150, Training Loss: 0.001488, Validation Loss: 0.003536, MAPE: 0.027684
Epoch 60/150, Training Loss: 0.000866, Validation Loss: 0.000342, MAPE: 0.006010
Epoch 70/150, Training Loss: 0.000896, Validation Loss: 0.001599, MAPE: 0.017848
Epoch 80/150, Training Loss: 0.000586, Validation Loss: 0.000442, MAPE: 0.007963
Epoch 90/150, Training Loss: 0.000575, Validation Loss: 0.000459, MAPE: 0.008361
Epoch 100/150, Training Loss: 0.000569, Validation Loss: 0.000580, MAPE: 0.009899
Epoch 110/150, Training Loss: 0.000544, Validation Loss: 0.000399, MAPE: 0.007681
Epoch 120/150, Training Loss: 0.000530, Validation Loss: 0.000475, MAPE: 0.008738
Epoch 130/150, Training L

/tmp/ipykernel_170358/2597736274.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/150, Training Loss: 0.022906, Validation Loss: 0.035143, MAPE: 0.072250
Epoch 20/150, Training Loss: 0.018919, Validation Loss: 0.001143, MAPE: 0.008346
Epoch 30/150, Training Loss: 0.008948, Validation Loss: 0.004844, MAPE: 0.023907
Epoch 40/150, Training Loss: 0.001995, Validation Loss: 0.004455, MAPE: 0.023953
Epoch 50/150, Training Loss: 0.001350, Validation Loss: 0.001308, MAPE: 0.010543
Epoch 60/150, Training Loss: 0.000869, Validation Loss: 0.001267, MAPE: 0.010207
Epoch 70/150, Training Loss: 0.000720, Validation Loss: 0.001143, MAPE: 0.009051
Epoch 80/150, Training Loss: 0.000732, Validation Loss: 0.001124, MAPE: 0.008682
Epoch 90/150, Training Loss: 0.000660, Validation Loss: 0.001169, MAPE: 0.009462
Epoch 100/150, Training Loss: 0.000650, Validation Loss: 0.001138, MAPE: 0.009182
Epoch 110/150, Training Loss: 0.000633, Validation Loss: 0.001132, MAPE: 0.009122
Epoch 120/150, Training Loss: 0.000628, Validation Loss: 0.001120, MAPE: 0.009016
Epoch 130/150, Training L

/tmp/ipykernel_170358/2597736274.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))


In [21]:
avg_mape = np.mean(list(mape_results.values()))
print(f"\nRata-rata MAPE dari model terbaik: {avg_mape:.6f}")


Rata-rata MAPE dari model terbaik: 0.020432
